
"""
Created on Sat Jul 20 16:31:20 2019

@author: jnikhil
- Nikhil J
"""

In [15]:
from urllib.request import urlopen  
from urllib import parse
from bs4 import BeautifulSoup
import requests
import requests.exceptions
from urllib.parse import urlsplit
import re
import json
import sys

In [12]:
class PageParser():           
        
        #This creates object for to be parsed for every url page
                    
    def getLinks(self, url):                # parsing each url page and getting links on the page
        
    
        self.links = []
        self.indomain = []
        self.baseUrl = url
        
        response = requests.get(url)  
        
        parts = urlsplit(url)
        base = "{0.netloc}".format(parts)
        strip_base = base.replace("www.", "")
     
        path = url[:url.rfind('/')+1] if '/' in parts.path else url
            
        soup = BeautifulSoup(response.text, "html.parser") 
        
        for link in soup.find_all('a'):    
        
            anchor = link.attrs["href"] if "href" in link.attrs else ''
                  
            if anchor.startswith('/'):
                newUrl = parse.urljoin(self.baseUrl, anchor)
                local = newUrl
                
            elif strip_base in anchor:
                newUrl = anchor
                local = newUrl
            
            elif not anchor.startswith('http'):
                newUrl = path + anchor
                local = newUrl
            else:
                newUrl = anchor           
        
            self.links = self.links + [newUrl]           # all links in domain and out domain 
            self.indomain = self.indomain + [local]      # only local links that need to be crawl based on depth 
            
        return (list(set(self.links)), list(set(self.indomain)))   #return both links on the page and links with same domain
    
    def getImage(self,url):                              #Parsing a url page for images
        
        self.images = []
    
        html = urlopen(url)
        bs = BeautifulSoup(html, 'html.parser')
       
        imgs = bs.find_all('img', {'src':re.compile('.')})
        
        for image in imgs: 
            self.images += [(image['src'])]    
        
        return self.images          #return images on the page'''



In [13]:
def  build_site_map(url, max_depth):  
    
    visited = []
    pages_to_visit = [url]
    visited_links = 0
    j_list = []
    '''
    This function takes two arguments url (from where start site map) and maximum number of pages to be visited 
    '''
    while visited_links < max_depth and pages_to_visit != []:
        visited_links = visited_links +1
        dict_={}    
        url = pages_to_visit[0]
        pages_to_visit = pages_to_visit[1:]
        images_on_page = []
        
        try:
        
            parser = PageParser()
            
            links,domain_links = parser.getLinks(url)            
            images = parser.getImage(url) 
            
            visited = visited + [url]
            pages_to_visit = pages_to_visit + domain_links            
            pages = set(pages_to_visit)-set(visited)  # remove visited urls          
            pages_to_visit = [ele for ele in pages_to_visit if ele in pages]
            
            images_on_page = images 
            
            dict_['page_url'] = url
            dict_['links'] = links
            dict_['images'] = images_on_page
            j_list.append(dict_)
            
        except:
            print("Error")
            
                
    return (json.dumps(j_list))      #returns json file 


In [14]:
url = "https://www.mozilla.org/en-US/"
max_depth = 3
json_data = build_site_map(url,max_depth)       
parsed = json.loads(json_data)
print(json.dumps(parsed, indent=2, sort_keys=False))  # prints output json file     

[
  {
    "page_url": "https://www.mozilla.org/en-US/",
    "links": [
      "https://www.telegraph.co.uk/technology/2019/06/14/firefox-browser-defeated-internet-explorer-plans-save-web/",
      "https://bugzilla.mozilla.org/",
      "https://developer.mozilla.org/docs/Tools/?utm_source=www.mozilla.org&utm_medium=referral&utm_campaign=footer&utm_content=developers",
      "https://getpocket.com/firefox_learnmore/?utm_source=www.mozilla.org&utm_medium=referral&utm_campaign=nav&utm_content=firefox",
      "https://www.mozilla.org/en-US/contact/",
      "https://www.mozilla.org/locales/",
      "https://www.mozilla.org/en-US/moss/",
      "https://download.mozilla.org/?product=firefox-msi-latest-ssl&os=win64&lang=en-US",
      "https://getpocket.com/@MozillaHQ",
      "https://support.mozilla.org/products/firefox/?utm_source=www.mozilla.org&utm_medium=referral&utm_campaign=nav&utm_content=firefox",
      "https://foundation.mozilla.org/?utm_source=www.mozilla.org&utm_medium=referral&utm_c